In [3]:
pip install pyspark


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [34]:
#initiate spark instance 
import pyspark
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
#Create SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("SparkApp") \
    .getOrCreate()

#spark.stop()
print("spark success")

spark success


In [2]:
#print current directory
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print(f"Current Working Directory: {current_directory}")


Current Working Directory: /Users/ruiiyoung/Downloads/MacAir2016_Desktop/IT stuff DIY +所有自己上的课/U St Andrews Data Science PGDip/5939data-driven system/assignment1/SparkA1


In [4]:
#part 1:1.load all csv in this same path
df= spark.read.format("csv").option("header", True).option("inferSchema", True).load("/Users/ruiiyoung/Downloads/MacAir2016_Desktop/IT stuff DIY +所有自己上的课/U St Andrews Data Science PGDip/5939data-driven system/assignment1/lab_dataset.csv")
#df.show()
display(df.schema) 

24/07/17 18:03:45 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


StructType([StructField('IncidentNumber', StringType(), True), StructField('DateOfCall', StringType(), True), StructField('CalYear', IntegerType(), True), StructField('TimeOfCall', TimestampType(), True), StructField('HourOfCall', IntegerType(), True), StructField('IncidentGroup', StringType(), True), StructField('StopCodeDescription', StringType(), True), StructField('SpecialServiceType', StringType(), True), StructField('PropertyCategory', StringType(), True), StructField('PropertyType', StringType(), True), StructField('AddressQualifier', StringType(), True), StructField('Postcode_full', StringType(), True), StructField('Postcode_district', StringType(), True), StructField('UPRN', DoubleType(), True), StructField('USRN', IntegerType(), True), StructField('IncGeo_BoroughCode', StringType(), True), StructField('IncGeo_BoroughName', StringType(), True), StructField('ProperCase', StringType(), True), StructField('IncGeo_WardCode', StringType(), True), StructField('IncGeo_WardName', Stri

In [74]:
#part1:2 Allow the user to search the dataset by the borough name, showing the total number of callouts in 2020.
# Filter the DataFrame to include only data for the year 2020
from pyspark.sql.functions import levenshtein


df_2020 = df.filter(col('CalYear') == 2020)
# Group the data by 'IncGeo_BoroughName' and sum the 'NumbCalls' column
df_Borough_call = df_2020.groupBy('IncGeo_BoroughName').sum('NumCalls')
df_Borough_call = df_Borough_call.withColumnRenamed('IncGeo_BoroughName', 'Borough Name')\
                                 .withColumnRenamed('sum(NumCalls)', 'Total number of callouts')

#df_Borough_call.show(n=33, truncate=False)
# Function to search by borough name
def search_borough():
    borough_name = input("Enter the borough name: ").strip().upper()
    #give error if input is not letters
    if not borough_name.isalpha():
        print ("Borough name should be a string.")
        return
    try:
        result = df_Borough_call.filter(col('Borough Name') == borough_name)
        # give error if the borough name is not in the dataset
        if result.count() == 0:
            # Calculate Levenshtein distance for each borough name
            distances = df_Borough_call.withColumn('distance', levenshtein(upper(col('Borough Name')), lit(borough_name)))
            
            # Find the closest match based on minimum distance
            closest_match = distances.select('Borough Name', 'distance').orderBy('distance').first()
            closest_borough_name = closest_match['Borough Name']

            
            print(f"No exact match found for '{borough_name}'. Did you mean '{closest_borough_name}'? ")
       
        else:
            result.show(truncate=False)
    
    except Exception as e:
        print(f"Error: {e}")
# Call the function to take user input and display the result
search_borough()

+------------+------------------------+
|Borough Name|Total number of callouts|
+------------+------------------------+
|HARROW      |2176                    |
+------------+------------------------+



In [6]:
#part1: 3  Given a range of hours (for example between 1-2 hours), show the user all incidents where the LFB spent this much time dealing with it.
# Define the hours of interest as a list
hours_of_interest = [1, 2]

# Filter DataFrame based on the HourOfCall and IncidentGroup conditions
df_time_spend = df.filter(
    (col("HourOfCall").isin(hours_of_interest)) &
    (col("IncidentGroup") != "False Alarm")
)

df_time_spend.show()

+---------------+-----------+-------+-------------------+----------+---------------+-------------------+--------------------+-----------------+--------------------+--------------------+-------------+-----------------+---------------+--------+------------------+--------------------+--------------------+---------------+-------------------+-------------------+---------+----------+---------------+----------------+-----------+------------+------+---------------------+--------------------------------+-------------------------------------+---------------------------------+--------------------------------------+-----------------------------+-----------------+---------+----------------+-----------------+--------+
| IncidentNumber| DateOfCall|CalYear|         TimeOfCall|HourOfCall|  IncidentGroup|StopCodeDescription|  SpecialServiceType| PropertyCategory|        PropertyType|    AddressQualifier|Postcode_full|Postcode_district|           UPRN|    USRN|IncGeo_BoroughCode|  IncGeo_BoroughName|    

In [7]:
#part1: 4 Calculate the number of incidents in 2021 where there were at least 2 fire pumps deployed.
# Filter DataFrame where PumpCount > 2
df_condition = df.filter((col("PumpCount") >2 ) & (col("CalYear") == 2021)) # >= 2 应该要用一个or

# Count distinct PumpCount values
IncidentNumber = df_condition.select("IncidentNumber").distinct().count()
print(f"Number of incidents is  {IncidentNumber} in 2021 where there were at least 2 fire pumps deployed ")


Number of incidents is  9077 in 2021 where there were at least 2 fire pumps deployed 


In [8]:
#part1:5.Which type of property had the most incident callouts during 2022?


from pyspark.sql.functions import col
from pyspark.sql import functions as F
# Filter the DataFrame to include only data for the year 2020
df_2022 = df.filter(col('CalYear') == 2022)
# property 
df_Property = df_2020.groupBy('PropertyType').agg(F.sum('NumCalls').alias('TotalNumCalls'))
df_Property = df_Property.orderBy(F.col('TotalNumCalls').desc())
df_Property.show(truncate=False) #这里printresult 真的要用print


+-----------------------------------------------------+-------------+
|PropertyType                                         |TotalNumCalls|
+-----------------------------------------------------+-------------+
|Purpose Built Flats/Maisonettes - 4 to 9 storeys     |17451        |
|House - single occupancy                             |14676        |
|Purpose Built Flats/Maisonettes - Up to 3 storeys    |11608        |
|Car                                                  |7377         |
|Self contained Sheltered Housing                     |6097         |
|Purpose Built Flats/Maisonettes - 10 or more storeys |4977         |
|Converted Flat/Maisonettes - 3 or more storeys       |4501         |
|Converted Flat/Maisonette - Up to 2 storeys          |3379         |
|Purpose built office                                 |3221         |
|Domestic garden (vegetation not equipment)           |2665         |
|Single shop                                          |2635         |
|Small refuse/rubbis

In [9]:
# Using an appropriate Python library, 
# draw a graph showing the trend of incident callouts on the road between 2020 and 2022. 
# Make sure to define how you interpret a road incident based on the dataset.

24/07/17 17:57:53 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
